In [1]:
#Import necessary packages.

from datetime import timedelta
from lark import Lark, Transformer
import os
import numpy as np
from random import shuffle
from time import time

In [2]:
#TPTP parser constructed using Lark.

tptp_parser = Lark(r"""
    ?tptp_file : tptp_input*
    ?tptp_input : annotated_formula | include

    ?annotated_formula : thf_annotated | tfx_annotated | tff_annotated| tcf_annotated | fof_annotated | cnf_annotated | tpi_annotated

    tpi_annotated : "tpi(" name "," formula_role "," tpi_formula annotations* ")."
    tpi_formula : fof_formula
    thf_annotated : "thf(" name "," formula_role "," thf_formula annotations* ")."
    tfx_annotated : "tfx("  name "," formula_role "," tfx_formula annotations* ")."
    tff_annotated : "tff(" name "," formula_role "," tff_formula annotations* ")."
    tcf_annotated : "tcf(" name "," formula_role "," tcf_formula annotations* ")."
    fof_annotated : "fof(" name "," formula_role "," fof_formula annotations* ")."
    cnf_annotated : "cnf(" name "," formula_role "," cnf_formula annotations* ")."
    annotations : "," source (optional_info)*

    formula_role : FORMULA_ROLE
    FORMULA_ROLE : "axiom" | "hypothesis" | "definition" | "assumption" | "lemma" | "theorem" | "corollary" | "conjecture"
                | "negated_conjecture" | "plain" | "type" | "fi_domain" | "fi_functors" | "fi_predicates" | "unknown" | LOWER_WORD

    thf_formula : thf_logic_formula | thf_sequent
    thf_logic_formula : thf_binary_formula | thf_unitary_formula | thf_type_formula | thf_subtype
    thf_binary_formula : thf_binary_pair | thf_binary_tuple | thf_binary_type

    thf_binary_pair : thf_unitary_formula thf_pair_connective thf_unitary_formula
    thf_binary_tuple : thf_or_formula | thf_and_formula | thf_apply_formula
    thf_or_formula : thf_unitary_formula "|" thf_unitary_formula | thf_or_formula "|" thf_unitary_formula
    thf_and_formula : thf_unitary_formula "&" thf_unitary_formula | thf_and_formula "&" thf_unitary_formula

    thf_apply_formula: thf_unitary_formula "@" thf_unitary_formula | thf_apply_formula "@" thf_unitary_formula

    thf_unitary_formula : thf_quantified_formula | thf_unary_formula | thf_atom | thf_conditional | thf_let | thf_tuple | "(" thf_logic_formula ")"

    thf_quantified_formula : thf_quantification thf_unitary_formula
    thf_quantification : thf_quantifier "[" thf_variable_list "] :"
    thf_variable_list : thf_variable ("," thf_variable)*
    thf_variable : thf_typed_variable | variable
    thf_typed_variable : variable ":" thf_top_level_type

    thf_unary_formula : thf_unary_connective "(" thf_logic_formula ")"
    thf_atom : thf_function | variable | defined_term | thf_conn_term

    thf_function : atom | functor "(" thf_arguments ")" | defined_functor "(" thf_arguments ")" | system_functor "(" thf_arguments ")"

    thf_conn_term : thf_pair_connective | assoc_connective | thf_unary_connective

    thf_conditional : "$ite(" thf_logic_formula "," thf_logic_formula "," thf_logic_formula ")"

    thf_let : "$let(" thf_unitary_formula "," thf_formula ")" | "$let(" thf_let_defns "," thf_formula")"
    thf_let_defns : thf_let_defn | "[" thf_let_defn_list "]"
    thf_let_defn_list : thf_let_defn ("," thf_let_defn)*
    thf_let_defn : thf_let_quantified_defn | thf_let_plain_defn
    thf_let_quantified_defn: thf_quantification "(" thf_let_plain_defn ")"
    thf_let_plain_defn: thf_let_defn_lhs ASSIGNMENT thf_formula
    thf_let_defn_lhs : constant | functor "(" fof_arguments ")" | thf_tuple

    thf_arguments : thf_formula_list

    thf_type_formula : thf_typeable_formula ":" thf_top_level_type | constant ":" thf_top_level_type
    thf_typeable_formula : thf_atom | "(" thf_logic_formula ")"
    thf_subtype : thf_atom "<<" thf_atom

    thf_top_level_type : thf_unitary_type | thf_mapping_type | thf_apply_type

    thf_unitary_type : thf_unitary_formula
    thf_apply_type : thf_apply_formula
    thf_binary_type : thf_mapping_type | thf_xprod_type | thf_union_type
    thf_mapping_type : thf_unitary_type ">" thf_unitary_type | thf_unitary_type ">" thf_mapping_type
    thf_xprod_type : thf_unitary_type "*" thf_unitary_type | thf_xprod_type "*" thf_unitary_type
    thf_union_type : thf_unitary_type "+" thf_unitary_type | thf_union_type "+" thf_unitary_type

    thf_sequent : thf_tuple "-->" thf_tuple | "(" thf_sequent ")"

    thf_tuple : "[" thf_formula_list? "]" | "{" thf_formula_list? "}"
    thf_formula_list : thf_logic_formula ("," thf_logic_formula)*

    logic_defn_rule : logic_defn_lhs assignment logic_defn_rhs
    logic_defn_lhs : logic_defn_value | thf_top_level_type | name | "$constants" | "$quantification" | "$consequence" | "$modalities"

    logic_defn_rhs : logic_defn_value | thf_unitary_formula
    logic_defn_value : LOGIC_DEFN_VALUE
    LOGIC_DEFN_VALUE : DEFINED_CONSTANT | "$rigid" | "$flexible" | "$constant" | "$varying" | "$cumulative" | "$decreasing" | "$local"
                    | "$global" | "$modal_system_K" | "$modal_system_T" | "$modal_system_D" | "$modal_system_S4" | "$modal_system_S5"
                    | "$modal_axiom_K" | "$modal_axiom_T" | "$modal_axiom_B" | "$modal_axiom_D" | "$modal_axiom_4" | "$modal_axiom_5"

    tfx_formula : tfx_logic_formula | thf_sequent
    tfx_logic_formula : thf_logic_formula

    tff_formula : tff_logic_formula | tff_typed_atom | tff_sequent
    tff_logic_formula : tff_binary_formula | tff_unitary_formula | tff_subtype
    tff_binary_formula : tff_binary_nonassoc | tff_binary_assoc
    tff_binary_nonassoc : tff_unitary_formula binary_connective tff_unitary_formula
    tff_binary_assoc : tff_or_formula | tff_and_formula
    tff_or_formula : tff_unitary_formula "|" tff_unitary_formula | tff_or_formula "|" tff_unitary_formula
    tff_and_formula : tff_unitary_formula "&" tff_unitary_formula | tff_and_formula "&" tff_unitary_formula
    tff_unitary_formula : tff_quantified_formula | tff_unary_formula | tff_atomic_formula | tff_conditional | tff_let | "(" tff_logic_formula ")"

    tff_quantified_formula : fof_quantifier "[" tff_variable_list "] :" tff_unitary_formula
    tff_variable_list : tff_variable ("," tff_variable)*
    tff_variable : tff_typed_variable | variable
    tff_typed_variable : variable ":" tff_atomic_type
    tff_unary_formula : "~" tff_unitary_formula | fof_infix_unary
    tff_atomic_formula : fof_atomic_formula
    tff_conditional : "$ite_f(" tff_logic_formula "," tff_logic_formula "," tff_logic_formula ")"
    tff_let : "$let_tf(" tff_let_term_defns "," tff_formula ")" | "$let_ff(" tff_let_formula_defns "," tff_formula ")"

    tff_let_term_defns : tff_let_term_defn | "[" tff_let_term_list "]"
    tff_let_term_list : tff_let_term_defn ("," tff_let_term_defn)*
    tff_let_term_defn : "! [" tff_variable_list "] :" tff_let_term_defn | tff_let_term_binding
    tff_let_term_binding : fof_plain_term "=" fof_term | "(" tff_let_term_binding ")"
    tff_let_formula_defns : tff_let_formula_defn | "[" tff_let_formula_list "]"
    tff_let_formula_list : tff_let_formula_defn ("," tff_let_formula_defn)*
    tff_let_formula_defn : "! [ "tff_variable_list "] :" tff_let_formula_defn | tff_let_formula_binding
    tff_let_formula_binding : fof_plain_atomic_formula "<=>" tff_unitary_formula | "(" tff_let_formula_binding ")"
    tff_sequent : tff_formula_tuple "-->" tff_formula_tuple | "(" tff_sequent ")"
    tff_formula_tuple : "[" [tff_formula_tuple_list] "]"
    tff_formula_tuple_list : tff_logic_formula ("," tff_logic_formula)*

    tff_typed_atom : untyped_atom ":" tff_top_level_type | "(" tff_typed_atom ")"
    tff_subtype : untyped_atom "<<" atom

    tff_top_level_type : tff_atomic_type | tff_mapping_type | tf1_quantified_type | "(" tff_top_level_type ")"
    tf1_quantified_type : "!> [" tff_variable_list "] :" tff_monotype
    tff_monotype : tff_atomic_type | "(" tff_mapping_type ")"
    tff_unitary_type :  tff_atomic_type | "(" tff_xprod_type ")"
    tff_atomic_type : type_constant | defined_type | type_functor "(" tff_type_arguments ")" | variable
    tff_type_arguments : tff_atomic_type ("," tff_atomic_type)*

    tff_mapping_type : tff_unitary_type ">" tff_atomic_type
    tff_xprod_type : tff_unitary_type "*" tff_atomic_type | tff_xprod_type "*" tff_atomic_type


    tcf_formula : tcf_logic_formula | tff_typed_atom
    tcf_logic_formula : tcf_quantified_formula | cnf_formula
    tcf_quantified_formula : "! [" tff_variable_list "] :" cnf_formula


    ?fof_formula : fof_logic_formula | fof_sequent
    ?fof_logic_formula : fof_binary_formula | fof_unitary_formula

    ?fof_binary_formula :  fof_binary_nonassoc | fof_binary_assoc

    ?fof_binary_nonassoc : fof_unitary_formula binary_connective fof_unitary_formula

    ?fof_binary_assoc : fof_or_formula | fof_and_formula
    fof_or_formula : fof_unitary_formula "|" fof_unitary_formula | fof_or_formula "|" fof_unitary_formula
    fof_and_formula  : fof_unitary_formula "&" fof_unitary_formula | fof_and_formula "&"  fof_unitary_formula

    ?fof_unitary_formula : fof_quantified_formula | fof_unary_formula | fof_atomic_formula | "(" fof_logic_formula ")"

    ?fof_quantified_formula : fof_quantifier "[" fof_variable_list "] :" fof_unitary_formula
    ?fof_variable_list : variable ("," variable)*
    fof_unary_formula : "~" fof_unitary_formula | fof_infix_unary

    fof_infix_unary : fof_term infix_inequality fof_term
    ?fof_atomic_formula : fof_plain_atomic_formula | fof_defined_atomic_formula | fof_system_atomic_formula
    ?fof_plain_atomic_formula : fof_plain_term
    ?fof_defined_atomic_formula : fof_defined_plain_formula | fof_defined_infix_formula
    ?fof_defined_plain_formula : fof_defined_plain_term | defined_proposition | defined_predicate "(" fof_arguments ")"
    ?fof_defined_infix_formula : fof_term defined_infix_pred fof_term

    ?fof_system_atomic_formula : fof_system_term

    ?fof_plain_term : constant | functor "(" fof_arguments ")"

    ?fof_defined_term : defined_term | fof_defined_atomic_term
    ?fof_defined_atomic_term : fof_defined_plain_term

    ?fof_defined_plain_term : defined_constant | defined_functor "(" fof_arguments ")"

    ?fof_system_term : system_constant | system_functor "(" fof_arguments ")"

    ?fof_arguments : fof_term ("," fof_term)*

    ?fof_term : fof_function_term | variable | tff_conditional_term | tff_let_term | tff_tuple_term
    ?fof_function_term : fof_plain_term | fof_defined_term | fof_system_term

    tff_conditional_term : "$ite_t(" tff_logic_formula "," fof_term "," fof_term ")"
    tff_let_term : "let_ft(" tff_let_formula_defns "," fof_term ")" | "$let_tt(" tff_let_term_defns ","fof_term ")"
    tff_tuple_term : "{" [fof_arguments] "}"


    fof_sequent : fof_formula_tuple "-->" fof_formula_tuple | "(" fof_sequent ")"
    ?fof_formula_tuple : "[" [fof_formula_tuple_list] "]"
    ?fof_formula_tuple_list : fof_logic_formula ("," fof_logic_formula)*


    cnf_formula : disjunction | "(" disjunction ")"
    disjunction : literal ("|" literal)*
    literal : fof_atomic_formula | "~" fof_atomic_formula | fof_infix_unary

    thf_quantifier : fof_quantifier | th0_quantifier | th1_quantifier

    th1_quantifier : TH1_QUANTIFIER
    TH1_QUANTIFIER : "!>" | "?*"
    th0_quantifier : TH0_QUANTIFIER
    TH0_QUANTIFIER : "^" | "@+" | "@-"
    thf_pair_connective : infix_equality | infix_inequality | binary_connective | assignment
    thf_unary_connective : "~" | TH1_UNARY_CONNECTIVE
    TH1_UNARY_CONNECTIVE : "!!" | "??" | "@@+" | "@@-" | "@="

    fof_quantifier : FOF_QUANTIFIER
    FOF_QUANTIFIER : "!" | "?"
    binary_connective : BINARY_CONNECTIVE
    BINARY_CONNECTIVE : "<=>" | "=>" | "<=" | "<~>" | "~|" | "~&"
    assoc_connective : ASSOC_CONNECTIVE
    ASSOC_CONNECTIVE : "&" | "|"

    assignment : ASSIGNMENT
    ASSIGNMENT : ":="

    type_constant : TYPE_CONSTANT
    TYPE_CONSTANT : TYPE_FUNCTOR
    type_functor : TYPE_FUNCTOR
    TYPE_FUNCTOR : ATOMIC_WORD
    defined_type : DEFINED_TYPE
    DEFINED_TYPE : ATOMIC_DEFINED_WORD | "$oType" | "$o" | "$iType" | "$i" | "$tType" | "$real" | "$rat" | "$int"

    system_type : SYSTEM_TYPE
    SYSTEM_TYPE : ATOMIC_SYSTEM_WORD

    atom : ATOM
    ATOM : UNTYPED_ATOM | DEFINED_CONSTANT
    untyped_atom : UNTYPED_ATOM
    UNTYPED_ATOM : CONSTANT | SYSTEM_CONSTANT
    defined_proposition : DEFINED_PROPOSITION
    DEFINED_PROPOSITION :  ATOMIC_DEFINED_WORD | "$true" | "$false"
    defined_predicate : DEFINED_PREDICATE
    DEFINED_PREDICATE : ATOMIC_DEFINED_WORD | "$distinct" | "$less" | "$lesseq" | "$greater" | "$greatereq" | "$is_int"
                        | "$is_rat" | "$box_P" | "$box_i" | "$box_int" | "$box" | "$dia_P" | "$dia_i" | "$dia_int" | "$dia"

    defined_infix_pred : infix_equality | assignment
    infix_equality : INFIX_EQUALITY
    INFIX_EQUALITY : "="
    infix_inequality : INFIX_INEQUALITY
    INFIX_INEQUALITY : "!="

    constant : CONSTANT
    CONSTANT : FUNCTOR
    functor : FUNCTOR
    FUNCTOR : ATOMIC_WORD
    system_constant : SYSTEM_CONSTANT
    SYSTEM_CONSTANT : SYSTEM_FUNCTOR
    system_functor : SYSTEM_FUNCTOR
    SYSTEM_FUNCTOR : ATOMIC_SYSTEM_WORD
    defined_constant : DEFINED_CONSTANT
    DEFINED_CONSTANT : DEFINED_FUNCTOR
    defined_functor : DEFINED_FUNCTOR
    DEFINED_FUNCTOR : ATOMIC_DEFINED_WORD |"$uminus" | "$sum" | "$difference" | "$product" | "$quotient" | "$quotient_e" | "$quotient_t" | "$quotient_f"
                    | "$remainder_e" | "$remainder_t" | "$remainder_f" | "$floor" | "$ceiling" | "$truncate" | "$round" | "$to_int" | "$to_rat" | "$to_real"
    ?defined_term : number | DISTINCT_OBJECT
    variable : VARIABLE
    VARIABLE : UPPER_WORD

    source : general_term | dag_source | internal_source | external_source | "[" sources "]"
    sources : source ("," source)*
    dag_source : name | inference_record
    inference_record : "inference(" inference_rule "," useful_info "," inference_parents ")"
    inference_rule : INFERENCE_RULE
    INFERENCE_RULE : ATOMIC_WORD

    inference_parents : "[" parent_list* "]"
    parent_list : parent_info ("," parent_info)*
    parent_info : source parent_details*
    parent_details : general_list
    internal_source : "introduced(" intro_type optional_info* ")"
    intro_type : "definition" | "axiom_of_choice" | "tautology" | "assumption"

    external_source : file_source | theory | creator_source
    file_source : "file(" FILE_NAME FILE_INFO* ")"
    FILE_INFO : "," NAME
    theory : "theory(" THEORY_NAME optional_info* ")"
    THEORY_NAME : "equality" | "ac"

    creator_source : "creator(" CREATOR_NAME optional_info* ")"
    CREATOR_NAME : ATOMIC_WORD

    optional_info : "," useful_info
    useful_info : general_list | "[" info_items* "]"
    info_items : info_item ("," info_item)*
    info_item : formula_item | inference_item | general_function

    formula_item : DESCRIPTION_ITEM  | IQUOTE_ITEM
    DESCRIPTION_ITEM : "description(" ATOMIC_WORD ")"
    IQUOTE_ITEM : "iquote(" ATOMIC_WORD ")"

    inference_item : inference_status | assumptions_record | new_symbol_record | refutation
    inference_status : "status(" STATUS_VALUE ")" | inference_info

    STATUS_VALUE : "suc" | "unp" | "sap" | "esa" | "sat" | "fsa" | "thm" | "eqv" | "tac" | "wec" | "eth" | "tau" | "wtc" | "wth" | "cax" | "sca" | "tca"
                | "wca" | "cup" | "csp" | "ecs" | "csa" | "cth" | "ceq" | "unc" | "wcc" | "ect" | "fun" | "uns" | "wuc" | "wct" | "scc" | "uca" | "noc"

    inference_info : inference_rule "(" ATOMIC_WORD "," general_list ")"

    assumptions_record : "assumptions([" name_list "])"

    refutation : "refutation(" file_source ")"

    new_symbol_record : "new_symbols(" ATOMIC_WORD ", [" new_symbol_list "])"
    new_symbol_list : principal_symbol ("," principal_symbol)*

    principal_symbol : functor | variable


    include : "include(" FILE_NAME formula_selection* ")."
    formula_selection : ",[" name_list "]"
    name_list : name ("," name)*

    general_term : general_data | general_data ":" general_term | general_list
    general_data : ATOMIC_WORD | general_function | variable | number | DISTINCT_OBJECT | formula_data | "bind(" variable "," formula_data ")"
    general_function : ATOMIC_WORD "(" general_terms ")"

    formula_data : "$thf(" thf_formula ")" | "$tff(" tff_formula ")" | "$fof(" fof_formula ")" | "$cnf(" cnf_formula ")" | "$fot(" fof_term ")"
    general_list : "[" general_terms? "]"
    general_terms : general_term ("," general_term)*

    name : NAME
    NAME : ATOMIC_WORD | INTEGER

    ATOMIC_WORD : LOWER_WORD | SINGLE_QUOTED

    ATOMIC_DEFINED_WORD : "$" LOWER_WORD
    ATOMIC_SYSTEM_WORD : "$$" LOWER_WORD
    number : INTEGER | RATIONAL | REAL

    FILE_NAME : SINGLE_QUOTED

    COMMENT : COMMENT_LINE | COMMENT_BLOCK
    COMMENT_LINE : "%" PRINTABLE_CHAR*
    COMMENT_BLOCK : "/*" NOT_STAR_SLASH? "*"+ "/"
    NOT_STAR_SLASH : ("^*"* "*"+ "^/*") ("^*")*

    SINGLE_QUOTED : "'" SQ_CHAR+ "'"

    DISTINCT_OBJECT : "\"" DO_CHAR* "\""

    UPPER_WORD : UPPER_ALPHA ALPHA_NUMERIC*
    LOWER_WORD : LOWER_ALPHA ALPHA_NUMERIC*

    REAL : SIGN? DECIMAL_FRACTION | SIGN? DECIMAL_EXPONENT
    RATIONAL : SIGN? DECIMAL "/" POSITIVE_DECIMAL
    INTEGER : SIGN? DECIMAL
    DECIMAL : ZERO_NUMERIC | POSITIVE_DECIMAL
    POSITIVE_DECIMAL : NON_ZERO_NUMERIC NUMERIC*           
    DECIMAL_EXPONENT : DECIMAL EXPONENT EXP_INTEGER | DECIMAL_FRACTION EXPONENT EXP_INTEGER
    DECIMAL_FRACTION : DECIMAL DOT_DECIMAL
    DOT_DECIMAL : "." NUMERIC+
    EXP_INTEGER : SIGN? NUMERIC+

    DO_CHAR : (/["\40"-"\41", "\43"-"\133", "\135"-"\176"]/ | "\\\\ \" \\\\")


    SQ_CHAR : (/["\40"-"\46", "\50"-"\133", "\135"-"\176"]/ | "\\\\ ' \\\\")

    SIGN : "+" | "-"
    EXPONENT : "E" | "e"
    ZERO_NUMERIC : "0"
    NON_ZERO_NUMERIC : "1" .. "9"
    NUMERIC : "0" .. "9"
    LOWER_ALPHA : "a" .. "z"
    UPPER_ALPHA : "A" .. "Z"
    ALPHA_NUMERIC : LOWER_ALPHA | UPPER_ALPHA | NUMERIC | "_"
    PRINTABLE_CHAR : /["\32"-"\126"]/

    VIEWABLE_CHAR : "\n"

    %import common.WS  
    %ignore WS
    %ignore COMMENT

    """, start='tptp_file')

In [3]:
class list_of_functions(Transformer):
    fof_annotated = lambda self, a: tuple([a[0], a[2]])
    fof_formula = lambda self, a: a
    fof_logic_formula = lambda self, a: a

    fof_binary_formula = lambda self, a : a

    fof_binary_nonassoc = lambda self, a : a

    fof_binary_assoc = lambda self, a : a
    fof_or_formula = lambda self, a : a
    fof_and_formula = lambda self, a : a

    fof_unitary_formula = lambda self, a : a

    fof_quantified_formula = lambda self, a : a
    fof_variable_list = lambda self, a : a
    fof_unary_formula = lambda self, a : a

    fof_infix_unary = lambda self, a : a
    fof_atomic_formula = lambda self, a : a
    fof_plain_atomic_formula = lambda self, a : a
    fof_defined_atomic_formula = lambda self, a : a
    fof_defined_plain_formula = lambda self, a : a
    fof_defined_infix_formula = lambda self, a : a

    fof_system_atomic_formula = lambda self, a : a

    fof_plain_term = lambda self, a : a

    fof_defined_term = lambda self, a : a
    fof_defined_atomic_term = lambda self, a : a

    fof_defined_plain_term = lambda self, a : a

    fof_system_term = lambda self, a : a

    fof_arguments = lambda self, a : a

    fof_term = lambda self, a : a
    fof_function_term = lambda self, a : a

    tff_conditional_term = lambda self, a : a
    tff_let_term = lambda self, a : a
    tff_tuple_term = lambda self, a : a


    fof_sequent = lambda self, a : a
    fof_formula_tuple = lambda self, a : a
    fof_formula_tuple_list = lambda self, a : a

    fof_quantifier = lambda self, a : []
    binary_connective = lambda self, a : []
    assoc_connective = lambda self, a : []

    assignment = lambda self, a : []

    type_constant = lambda self, a : a[0][:]
    type_functor = lambda self, a : a[0][:]
    defined_type = lambda self, a : a[0][:]

    system_type = lambda self, a : a[0][:]

    atom = lambda self, a : a
    untyped_atom = lambda self, a : a
    defined_proposition = lambda self, a : []
    defined_predicate = lambda self, a : a[0][:]

    defined_infix_pred = lambda self, a : []
    infix_equality = lambda self, a : []
    infix_inequality = lambda self, a : []

    constant = lambda self, a : a[0][:]
    functor = lambda self, a : a[0][:]
    system_constant = lambda self, a : a[0][:]
    system_functor = lambda self, a : a[0][:]
    defined_constant = lambda self, a : a[0][:]
    defined_functor = lambda self, a : a[0][:]
    defined_term = lambda self, a : a
    variable = lambda self, a : []

    name = lambda self, a : a[0][:]

    number = lambda self, a : []

In [4]:
def flatten_list(L):
    flattened_L = []
    for l in L:
        if isinstance(l, list):
            flattened_L += flatten_list(l)
        else:
            flattened_L.append(l)
    return flattened_L

In [5]:
def extract_functions(tptp_file):
    tree = tptp_parser.parse(tptp_file)
    list_of_fun = list_of_functions().transform(tree)
    premise_name = list_of_fun[0]
    premise_functions = flatten_list(list_of_fun[1])
    
    return premise_name, premise_functions

In [7]:
#Create functional signatures for premises.

start_time = time()

Conjectures_by_functions = {}
Axioms_by_functions = {}
Useful_axioms = {}
Useless_axioms = {}

current_directory = os.getcwd()
os.chdir(current_directory+"/nndata")

for file in os.listdir():
    with open(file, "r") as current_file:
        problem = current_file.read()
    Premises = problem.split("\n")[:-1]
    conjecture = Premises[0][2:]
    conjecture_name, conjecture_functions = extract_functions(conjecture)
    Conjectures_by_functions[conjecture_name] = conjecture_functions
    
    Useful_axioms[conjecture_name] = []
    Useless_axioms[conjecture_name] = []
    
    for axiom in Premises[1:]:
        axiom_name = axiom[6:].split(', axiom, ')[0] #To aviod lenghty parsing the same axiom multiple times.
        if axiom.startswith('+'):
            Useful_axioms[conjecture_name].append(axiom_name)
        elif axiom.startswith('-'):
            Useless_axioms[conjecture_name].append(axiom_name)
            
        if not axiom_name in Axioms_by_functions.keys():
            Axioms_by_functions[axiom_name] = extract_functions(axiom[2:])[1]

os.chdir(current_directory)

end_time = time()

print('Formating time:', str(timedelta(seconds = end_time-start_time)), 'hours.')

Sorted_conjectures = sorted(list(Conjectures_by_functions.keys()))
Sorted_axioms = sorted(list(Axioms_by_functions.keys()))
Sorted_premises = Sorted_conjectures + Sorted_axioms

print('Total number of unique conjectures:', str(len(Sorted_conjectures)) + '.')
print('Total number of unique axioms:', str(len(Sorted_axioms)) + '.')
print('Total number of unique premises:', str(len(Sorted_premises)) + '.')

Total number of unique conjectures: 32524.
Total number of unique axioms: 69918.
Total number of unique premises: 102442.


In [10]:
Conjectures_functions = set()
Axioms_functions = set()

for functions in Conjectures_by_functions.values():
    Conjectures_functions |= set(functions)

for functions in Axioms_by_functions.values():
    Axioms_functions |= set(functions)

Functions = sorted(Conjectures_functions | Axioms_functions)

print('Total number of functions in the scope of axioms:', str(len(Axioms_functions)) + '.')
print('Total number of functions in the scope of conjectures:', str(len(Conjectures_functions))+ '.')
print('Total numbber of unique functions:', str(len(Functions)) + '.')

Total number of functions in the scope of axioms: 13331.
Total number of functions in the scope of conjectures: 8369.
Total numbber of unique functions: 13335.


In [11]:
X = np.zeros((len(Sorted_premises), len(Functions)), dtype = 'int16')

for n in range(len(Functions)):
    for m in range(len(Sorted_conjectures)):
        X[m, n] = Conjectures_by_functions[Sorted_conjectures[m]].count(Functions[n])
    for m in range(len(Sorted_axioms)):
        X[m + len(Sorted_conjectures), n] = Axioms_by_functions[Sorted_axioms[m]].count(Functions[n])

np.save('Fun', X)

In [12]:
print(Functions)

['0', '1', '10', '100', '1024', '104', '105', '11', '112', '118', '119', '12', '120', '124', '126', '127', '128', '13', '136', '14', '15', '159', '16', '17', '18', '19', '197', '2', '20', '2047', '2048', '209', '21', '211', '216', '22', '23', '24', '241', '25', '251', '255', '256', '257', '26', '27', '28', '29', '2985', '3', '30', '3072', '31', '3125', '316', '32', '33', '34', '35', '36', '365', '366301321', '369', '37', '38', '39', '4', '40', '4001', '4096', '41', '414', '419', '42', '423', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '64864', '65536', '65537', '6561', '66', '67', '672', '7', '72', '78', '8', '80', '81', '83', '84', '841', '89', '9', '96', 'a_0_0_abcmiz_1', 'a_0_0_arytm_0', 'a_0_0_arytm_2', 'a_0_0_arytm_3', 'a_0_0_asympt_1', 'a_0_0_borsuk_6', 'a_0_0_complex1', 'a_0_0_comput_1', 'a_0_0_cqc_the1', 'a_0_0_dickson', 'a_0_0_dtconstr', 'a_0_0_euler_1', 'a_0_0_fdiff_7', 'a_0_0_fd

In [ ]:
Conjectures_vectorized = np.load('Conj_tokens.npy')
Axioms_vectorized = np.load('Ax_tokens.npy')

print(Conjectures_vectorized.shape)
print(Axioms_vectorized.shape)

In [ ]:
data = []

for m in range(len(Sorted_conjectures)):
    conjecture = Conjectures_vectorized[m]
    for axiom in Useful_axioms[Sorted_conjectures[m]]:
        n = Sorted_axioms.index(axiom)
        ax = Axioms_vectorized[n]
        data.append((np.concatenate((conjecture, ax), axis = 0), 1))
    for axiom in Useless_axioms[Sorted_conjectures[m]]:
        n = Sorted_axioms.index(axiom)
        ax = Axioms_vectorized[n]
        data.append((np.concatenate((conjecture, ax), axis = 0), 0))

print('Number of positive and negative (conjecture, axiom) pairs:', str(len(data)) + '.')

In [ ]:
shuffle(data)
training_slice = int(len(data)*0.9)

input_data = [datum[0] for datum in data]
output_labels = [datum[1] for datum in data]

X_train = np.array(input_data[:training_slice])
X_test = np.array(input_data[training_slice:])
Y_train = np.array(output_labels[:training_slice])
Y_test = np.array(output_labels[training_slice:])

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

np.save('Training_data', X_train)
np.save('Test_data', X_test)
np.save('Training_labels', Y_train)
np.save('Test_labels', Y_test)